In [2]:
using Pkg
Pkg.activate(".")

using Plots
using ASE
using JuLIP

┌ Info: Precompiling ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c]
└ @ Base loading.jl:1317
┌ Info: Skipping precompilation since __precompile__(false). Importing ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c].
└ @ Base loading.jl:1025


In [3]:


function unflatten(flat_array; shape=(40, 40, 40))
    return permutedims(reshape(flat_array, shape), [3, 2, 1])
end

function integrate(data; step=40)
    output=zeros(0)
    for idx in 1:step
        slice = data[idx,:,:]
        append!(output, sum(slice))
    end
    return output
end

function convert_axis(origin, axes; which_axis=1, dimensions=(40, 40, 40))
    range = Array(1:dimensions[which_axis])
    # cannot do shearing 
    scaled = range * axes[which_axis][which_axis]
#     println(origin[which_axis])
    shifted = [x + origin[which_axis] - axes[which_axis][which_axis] for x in scaled]
    return shifted
end

function read_file(fliename)

    file = readlines(filename)

    # two lines of comments
    # third line: number of atoms (int) and origin (3 x float)
    n_atoms = parse(Int64, split(file[3])[1])
    origin = parse.(Float64, split(file[3])[2:4])

    # fourth-sixth lines - number of voxels along each axis and the axis. 
    # Take only number of voxels (for now?)    
    dens_shape = Tuple([parse(Int64, split(file[i])[1]) for i in 4:6])
    axes = [parse.(Float64, split(file[i])[2:4]) for i in 4:6]

    # `n_atoms` of geometry data
    numbers = [parse(Int64, split(file[i])[1]) for i in 7:6+n_atoms]
    pos = [parse.(Float64, split(file[i])[3:5]) for i in 7:6+n_atoms]
    atoms = Atoms(X=pos, Z=numbers)

    # density data starts after `n_atoms` lines of geometry data
    flat_values=zeros(0)
    for line in file[7+n_atoms:end]
        numbers = parse.(Float64, split(line))
        append!(flat_values, numbers)
    end

    density = unflatten(flat_values, shape=dens_shape);

    return origin, axes, atoms, density, dens_shape

end

read_file (generic function with 1 method)

In [6]:
filename = "HC30H.singlet_opt.uks_def2-svp_opt.dlpno-ccsd_cc-pvdz.singlet.eldens.200x200x200.cube"

pic_fname = replace(filename, ".cube" => ".pdf")

title = split(filename, "/")[end]
origin, axes, atoms, el_density, dens_shape = read_file(filename)
integral = integrate(el_density, step=dens_shape[1])


x_vals = convert_axis(origin, axes, dimensions=dens_shape)
at_pos = [atoms.X[i][1] for i in 1:32 if atoms.Z[i] == 6]
plot(x_vals, integral, label="xy-integrated density", lw=2)
# ylims!(0, 6)
# title!(title, titlefontsize=8)
vline!(at_pos, label="Carbon atoms' positions")
xlabel!("z coordinate")
# ylabel!("")
# savefig("density_example.pdf")
savefig(pic_fname)

In [9]:
readdir("../test_plots")

7-element Vector{String}:
 "eldens.cube"
 "out.dat"
 "plot_template_density_cross_section.vmd"
 "plot_template_isosurfaces.vmd"
 "plot_template_isosurfaces_eldens.vmd"
 "plot_template_isosurfaces_spindens.vmd"
 "spindens.cube"